In [61]:
import numpy as np
import pandas as pd
import re
import math
import time

In [62]:
FILENAME="input.txt"
#FILENAME="test.txt"
DIAG=False
DIAG2 = False
MY_INF = -np.inf

In [63]:
def parseFile(file_name):
    with open(file_name) as file:
        lines = file.readlines()
    
    bricks=[]
    
    for line in lines:
        if line[-1] == "\n": line = line[:-1]
        str_re = "(\d+),(\d+),(\d+)~(\d+),(\d+),(\d+)"
        mmm = re.match(str_re,line)
        brick_st = np.array((int(mmm[1]),int(mmm[2]),int(mmm[3])))
        brick_en = np.array((int(mmm[4]),int(mmm[5]),int(mmm[6])))
        brick_len = brick_en-brick_st
        bricks += [(brick_st,brick_len)]
    
    return bricks
    

In [64]:
def constructBrickMap(bricks):
    brick_ends = [x for x,_ in bricks]+[x+ll for x,ll in bricks]
    
    min_dim = np.minimum.reduce(brick_ends)
    max_dim = np.maximum.reduce(brick_ends)
    
    min_dim[2] = 1
    
    brick_map = np.ones(max_dim-min_dim+np.array((1,1,2)))*MY_INF
    
    return brick_map

In [65]:
def fillBrickMap(bricks,brick_map):
    shX,shY,shZ = brick_map.shape
    
    for x in range(0,shX):
        for y in range(0,shY):
            brick_map[x,y,0]=-1
    
    for i,(br_st,br_len) in enumerate(bricks):

        br_min = np.minimum.reduce([br_st,br_st+br_len])
        br_max = np.maximum.reduce([br_st,br_st+br_len])
        if DIAG: print(">>",br_min,br_max)
        
        for x in range(br_min[0],br_max[0]+1):
            for y in range(br_min[1],br_max[1]+1):
                for z in range(br_min[2],br_max[2]+1):
                    brick_map[x][y][z]=i
    return brick_map

In [66]:
def constructAndFillMap(bricks):
    
    brick_map = constructBrickMap(bricks)
    brick_map = fillBrickMap(bricks,brick_map)
    
    return brick_map

In [67]:
def canMoveDownAbove(st_x,st_y,st_z,bricks_arr):
    
    shX,shY,shZ = bricks_arr.shape
    
    found=False
    # first find a brick piece above, if it exists of course
    for z in range(st_z+1,shZ):
        if bricks_arr[st_x,st_y,z] != MY_INF:
            # we found the tile
            found = True
            break
    
    if not found:    
        # piece not found (column above is empty), we won't be able to change anything 
        return False
    
    # !!! we know that bricks change only in 1 dimension, from examining the input data
    el = bricks_arr[st_x,st_y,z]
    
    canMove = True
    for x in range(0,shX):
        if canMove:
            for y in range(0,shY):
                if bricks_arr[x,y,z] == el:
                    if bricks_arr[x,y,z-1]!=MY_INF:
                        canMove = False
                        break
        else:
            break
    return canMove

In [68]:
def moveDown(st_x,st_y,st_z,bricks_arr):
    shX,shY,shZ = bricks_arr.shape
    
    found = False
    
    # first find a brick piece above, if it exists of course
    for z in range(st_z+1,shZ):
        if bricks_arr[st_x,st_y,z] != MY_INF:
            # we found the tile
            found = True
            break
    
    if not found:    
        # piece not found (column above is empty), we won't be able to change anything 
        return bricks_arr
    
    # !!! we know that bricks change only in 1 dimension, from examining the input data
    el = bricks_arr[st_x,st_y,z]
    
    canMove = True
    for x in range(0,shX):
        if canMove:
            for y in range(0,shY):
                if bricks_arr[x,y,z] == el:
                    if bricks_arr[x,y,z-1]!=MY_INF:
                        canMove = False
                        break
        else:
            break
            
    if canMove:
        z_cur = z
        changed = True
        if changed:
            changed = False
            for x in range(0,shX):
                for y in range(0,shY):
                    if bricks_arr[x,y,z] == el:
                        tmp = bricks_arr[x,y,z-1]
                        bricks_arr[x,y,z-1]= el
                        bricks_arr[x,y,z]= tmp
                        changed=True
            z_cur +=1
    
    # then find brick pieces on the same level - and check if they have void underneath
    
    # if so, move all of them - and stack above them - 1 down
    return bricks_arr        
            
        
    
     
    

In [69]:
def settleOneStep(bricks_settled,z_to_start):
    
    shX,shY,shZ = bricks_settled.shape
        
    for z in range(z_to_start,shZ):
        if np.all(bricks_settled[:,:,z]==MY_INF):
            #remove row
            bricks_settled=np.delete(bricks_settled,z,2)
            return bricks_settled,True,z
        else:
            for x in range(0,shX):
                for y in range(0,shY):
                    if bricks_settled[x,y,z]==MY_INF:
                        # find next brick above
                        # check if can move down
                        if canMoveDownAbove(x,y,z,bricks_settled):
                            bricks_settled = moveDown(x,y,z,bricks_settled)
                            return bricks_settled,True,z
    
    return bricks_settled,False,shZ


In [70]:
def settleBricksDown(arrBrickMap):
    bricks_settled = arrBrickMap.copy()
    changed = True
    z_to_start = 1
    while changed:
        if DIAG: print("*")
        bricks_settled,changed,z_to_start = settleOneStep(bricks_settled,z_to_start)
    
    return bricks_settled

In [71]:
def trimFinalRows(arrBrickMap):
    _,_,shZ = arrBrickMap.shape
    for z in range(shZ-1,-1,-1):
        if np.all(arrBrickMap[:,:,z]==MY_INF):
            arrBrickMap = arrBrickMap[:,:,:z]
        else:
            break
    return arrBrickMap

In [72]:
st_time = time.time()

bricks = parseFile(FILENAME)
print("Number of bricks:",len(bricks))
arrBrickMap = constructAndFillMap(bricks)
print("Shape of map:",arrBrickMap.shape)
if DIAG2: print(arrBrickMap.shape)

print("To construct: ",time.time()-st_time)
st_time = time.time()
arrBrickMap_settle = settleBricksDown(arrBrickMap)
arrBrickMap_settle

print("To settle: ",time.time()-st_time)
print("starting the checks")

#countOfBricks = countBricksOKToRemove(arrBrickMap_settle)
#print("Can remove {} bricks".format(countOfBricks))

Number of bricks: 1388
Shape of map: (10, 10, 326)
To construct:  0.07647442817687988
To settle:  148.47327280044556
starting the checks


use arrBrickMap_settle

In [73]:
def checkIfOkToRemove(brickNum,arrBrickMap):
    ind_x,ind_y,ind_z =np.where(arrBrickMap == brickNum) 
    
    
    arrCopy = arrBrickMap.copy()
    arrCopy[arrCopy==brickNum]=MY_INF
    
    for i,_ in enumerate(ind_x):
        x=ind_x[i]
        y=ind_y[i]
        z=ind_z[i]
        
        if canMoveDownAbove(x,y,z,arrCopy):
            return False
    
    return True
        
        # question is really whether an element directly above is supported by anything else
        
    
    

In [74]:
def countBricksOKToRemove(arrBrickMap):
    maxBrick = np.max(arrBrickMap)
    res = 0
    for brick in range(0,int(maxBrick)+1):
        if checkIfOkToRemove(brick,arrBrickMap_settle):
            if DIAG2: print("Brick {} ({} in example) OK to remove".format(brick,brick+1))
            res+=1
    return res
        

In [ ]:
# Bricks that can be removed without upsetting the proverbial cart
resA = countBricksOKToRemove(arrBrickMap_settle)
print("Result part A:",resA)

In [76]:
def countMisplacedBricks(arrCopy,arrCopy_settled):
    res = 0
    shX,shY,shZ = np.minimum.reduce([arrCopy.shape,arrCopy_settled.shape])
    arrCopy1 = arrCopy[:shX,:shY,:shZ].flatten()
    arrCopy_settled1 = arrCopy_settled[:shX,:shY,:shZ].flatten()
    #same_bricks = arrCopy1[arrCopy1==arrCopy_settled1]
    different_bricks = np.unique(arrCopy_settled1[arrCopy1!=arrCopy_settled1])
    #print(different_bricks)
    mask = ((different_bricks==MY_INF) | (different_bricks==-1))
    real_different = np.delete(different_bricks,mask,0)
    
    
    
    return len(real_different)

In [77]:
def countBricksFalling_single(brickNum,arrBrickMap):
    
    arrCopy = arrBrickMap.copy()
    arrCopy[arrCopy==brickNum]=MY_INF
    arrCopy_settled = settleBricksDown(arrCopy)
    res = countMisplacedBricks(arrCopy,arrCopy_settled)
    #print(" Brick {} ({}) = {}".format(brickNum,brickNum+1,res))
    return res

In [78]:
#Sum of bricks that would fall
def countBricksFalling(arrBrickMap):
    maxBrick = np.max(arrBrickMap)
    res = 0
    for brick in range(0,int(maxBrick)+1):
        if not checkIfOkToRemove(brick,arrBrickMap_settle):
            res+=countBricksFalling_single(brick,arrBrickMap)
    return res
resB = countBricksFalling(arrBrickMap_settle)
print("Result part B",resB)     

75784

In [79]:
arrBrickMap_settle.shape

(10, 10, 161)

# Testing things out

In [ ]:
len(np.unique(arrBrick_INPUT))-2

In [ ]:
a = np.unique(arrBrick_INPUT)
mask = ((a==MY_INF) | (a==-1))
aa = np.delete(a,mask,0)
len(aa)

In [ ]:
arrBrick_INPUT.flatten()